In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24374339
paper_name = 'prescott_simmonds_2014' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_csv('raw_data/hits.txt', header=None, names=['orf'])

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 7 x 1


In [10]:
original_data.head()

,orf
0,YOR207C
1,YDL150W
2,YJR065C
3,YPL143W
4,YLR033W


In [11]:
original_data['orf'] = original_data['orf'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf]
Index: []


In [15]:
original_data['data'] = -1

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(7, 1)

# Load & process tested strains

In [20]:
tested = pd.read_excel('raw_data/Copy of YSC1057_Essential_het_diploid_obs_v5.0.xlsx', 
                       sheet_name='Essential Heterozygous Diploid')

In [21]:
tested.head()

,RECORD #,ORF,Strain,PLATE,ROW,COLUMN,Comments
0,20362.0,YAL043C,BY4743,401.0,A,2.0,NaN
1,20365.0,YAL041W,BY4743,401.0,A,3.0,met- did not mate
2,20368.0,YAL038W,BY4743,401.0,A,4.0,NaN
3,20372.0,YAL034W-a,BY4743,401.0,A,5.0,NaN
4,20375.0,YAL033W,BY4743,401.0,A,7.0,NaN


In [22]:
tested['orf'] = tested['ORF'].astype(str)

In [23]:
tested['orf'] = clean_orf(tested['orf'])

In [24]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

             RECORD #  ORF Strain  PLATE  ROW  COLUMN Comments  orf
index_input                                                        
1108              NaN  NaN    NaN    NaN  NaN     NaN      NaN  NAN


In [26]:
tested = tested.loc[t,:]

In [27]:
tested_orfs = np.unique(tested['orf'].values)

In [28]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [29]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [30]:
data = original_data.copy()

In [31]:
dataset_ids = [15988]
datasets = datasets.reindex(index=dataset_ids)

In [32]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [33]:
data.head()

dataset_id,15988
data_type,value
orf,
YAL001C,0
YAL003W,0
YAL025C,0
YAL032C,0
YAL033W,0


## Subset to the genes currently in SGD

In [34]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 2


In [35]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,15988
,data_type,value
gene_id,orf,
1,YAL001C,0
3,YAL003W,0
23,YAL025C,0
30,YAL032C,0
31,YAL033W,0


# Normalize

In [36]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [37]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [38]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      15988       
data_type       value valuez
gene_id orf                 
1       YAL001C     0    0.0
3       YAL003W     0    0.0
23      YAL025C     0    0.0
30      YAL032C     0    0.0
31      YAL033W     0    0.0

# Print out

In [39]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [40]:
from IO.save_data_to_db3 import *

In [41]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 24374339...
Inserting the new data...


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]

Updating the data_modified_on field...
